In [ ]:
import numpy as np
import tensorflow as tf
import pickle
import glob

%load_ext autoreload
%autoreload 2
%aimport likelihood

import likelihood as lh
import ddm

import matplotlib.pyplot as plt
import matplotlib as mpl

# Plotting settings 
mpl.rcParams['font.size'] = 18
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = [13,8]

In [ ]:
location_files = glob.glob('inference_results/*.p')

In [ ]:
location_files[0]

In [ ]:
location_probs = {}
for file in location_files:
    with open(file, 'rb') as f:
        fname = file.split('/')[-1]
        location_probs[fname] = pickle.load(f)

In [ ]:
len(location_probs.keys())

In [ ]:
keys = list(location_probs.keys())
location_probs[keys[0]].shape

plt.imshow(location_probs[keys[0]][60,:,:])

In [ ]:
def compile_train(location_dict, n_samples=100, basis_length=5):
    basis = []
    test = []
    for i in range(n_samples):
        loc = np.random.choice(list(location_dict.keys()))
        
        start = np.random.choice(np.arange(location_dict[loc].shape[0]-basis_length-1))
        basis += [location_dict[loc][start:start+basis_length]]
        test += [location_dict[loc][start+basis_length+1]]
    basis = np.stack(basis)
    basis = tf.convert_to_tensor(basis, dtype=float)
    basis = tf.reshape(basis, [basis.shape[0], basis.shape[1], basis.shape[2], basis.shape[3], 1])
    
    test = np.stack(test)
    test = tf.convert_to_tensor(test, dtype=float)
    test = tf.reshape(test, [test.shape[0], 1, test.shape[1], test.shape[2], 1])
    
    return basis, test         

In [ ]:
basis_length = 10

X, y = compile_train(location_probs, n_samples=500, basis_length=basis_length) 
X.shape, y.shape

In [ ]:
tf.config.list_physical_devices()

In [ ]:
normalization = 'none'
model = ddm.fit_observation(X, y, num_steps=2000, learning_rate=0.001,
                            reg=0.01, normalization=normalization)

In [ ]:
# might want to mess with reg until we get stable values here
model['gamma'].numpy().ravel()

In [ ]:
model['rmse']

In [ ]:
plt.plot(np.arange(len(model['loss'])), model['loss'])
ax=plt.gca()
ax.set_yscale('log')
plt.show()

In [ ]:
pred = ddm.predict(model['gamma'], X, normalization=normalization,
                   mean=model['mean'], std=model['std'])
if normalization == 'global' or normalization == 'local':
    true, mean, std = ddm.normalize(y, method=normalization,
                                    mean=model['mean'], std=model['std'])
    true = true[:, 0, :, :, :]
else:
    true = y[:, 0, :, :, :]

print('total loss', tf.math.reduce_sum(tf.abs(pred-true)))

for i in range(10):
    p = pred[i, :, :, 0]
    plt.imshow(p)
    plt.colorbar()
    plt.show()

    plt.imshow(true[i, :, :, 0])
    plt.colorbar()
    plt.show()
    
    plt.imshow(abs(pred[i, :, :, 0] - true[i, :, :, 0]) / model['rmse'])
    plt.colorbar()
    plt.show()
    
    break

In [ ]:
X.shape, y.shape

In [ ]:
hot_map.shape, hot_score.shape

In [ ]:
hot_map = ddm.hot_detect(model['gamma'], basis=X, test=y, rmse=model['rmse'],
                         normalization=normalization, mean=model['mean'], std=model['std'],
                         reduce=False)
hot_score = ddm.hot_detect(model['gamma'], basis=X, test=y, rmse=model['rmse'],
                         normalization=normalization, mean=model['mean'], std=model['std'],
                         reduce=True)

In [ ]:
order = np.argsort(hot_score)[::-1]
for i in order[:5]:
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
    f.suptitle(f'hot score: {hot_score[i]: .2e}')
    f.tight_layout()
    
    pre = tf.math.reduce_sum(X[i], axis=[0, -1])
    ax1.imshow(pre) #vmin=0, vmax=1)
    ax1.set_title('pre')
    #plt.colorbar(im, ax=ax1, shrink=0.3)
    
    post = tf.math.reduce_sum(y[i], axis=[0, -1])
    ax2.imshow(post)# vmin=0, vmax=1)
    ax2.set_title('post')
    
    im = ax3.imshow(hot_map[i, 0, :, :, 0], vmin=0, vmax=5)
    ax3.set_title('anomaly')
    plt.colorbar(im, ax=ax3, shrink=0.3)
    
    plt.subplots_adjust(top=1.25)
    plt.show()
    
for i in order[-5:]:
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
    f.suptitle(f'hot score: {hot_score[i]: .2e}')
    f.tight_layout()
    
    pre = tf.math.reduce_sum(X[i], axis=[0, -1])
    ax1.imshow(pre)#, vmin=0, vmax=1)
    ax1.set_title('pre')
    
    post = tf.math.reduce_sum(y[i], axis=[0, -1])
    ax2.imshow(post)#, vmin=0, vmax=1)
    ax2.set_title('post')
    
    im = ax3.imshow(hot_map[i, 0, :, :, 0], vmin=0, vmax=5)
    ax3.set_title('anomaly')
    plt.colorbar(im, ax=ax3, shrink=0.3)
    
    plt.subplots_adjust(top=1.25)
    plt.show()

In [ ]:
hot_score = ddm.hot_detect(model['gamma'], basis=X, test=y, rmse=model['rmse'],
                         normalization=normalization, mean=model['mean'], std=model['std'],
                         reduce=True)
plt.hist(hot_score.numpy(), bins=50)
plt.show()

In [ ]:
location_probs['loc_0386.p'].shape, X.shape

In [ ]:
def expand_loc_series(loc_array, timestep=10):
    steps = np.arange(len(loc_array)-timestep-1, step=1)
    X = []
    y = []
    for step in steps:
        X += [loc_array[step:step+timestep]]
        y += [loc_array[step+timestep+1]]
      
    X = np.stack(X)
    X = tf.convert_to_tensor(X, dtype=float)
    X = tf.reshape(X, [X.shape[0], X.shape[1], X.shape[2], X.shape[3], 1])
    
    y = np.stack(y)
    y = tf.convert_to_tensor(y, dtype=float)
    y = tf.reshape(y, [y.shape[0], 1, y.shape[1], y.shape[2], 1])
    
    return X, y

In [ ]:
X_test, y_test = expand_loc_series(location_probs['loc_0386.p'])

hot_score = ddm.hot_detect(model['gamma'], basis=X_test, test=y_test, rmse=model['rmse'],
                           normalization=normalization, mean=model['mean'], std=model['std'],
                           reduce=True)

plt.scatter(np.arange(hot_score.shape[0]), hot_score.numpy())
plt.show()

timestep = 10
for i in np.arange(len(location_probs['loc_0386.p'])-timestep-1, step=timestep):
    plt.imshow(np.log(location_probs['loc_0386.p'][i]))
    plt.title(i)
    plt.show()

In [ ]:
from tqdm.notebook import tqdm 

In [ ]:
results = {}
timestep = 10

for k, v in tqdm(location_probs.items(), total=len(location_probs)):
    
    X_test, y_test = expand_loc_series(v, timestep=timestep)
    hot_score = ddm.hot_detect(model['gamma'], basis=X_test, test=y_test, rmse=model['rmse'],
                           normalization=normalization, mean=model['mean'], std=model['std'],
                           reduce=True)
    
    results[k] = hot_score

In [ ]:
with open('ddm_results.p', 'wb') as f: 
    pickle.dump(results, f)